In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pandas as pd
from collections import Counter
import numpy as np
import sys
import matplotlib.pyplot as plt

# LOAD DATA

In [2]:
df_usr_view_pay = pd.read_csv('dataset/df_view_pay_with_month_week_aqi_weather.csv', index_col=[0, 1])
df_usr_view_pay

view_cnt  pay_cnt  weekday  month  aqi  weather  wind  \
shop_id time_stamp                                                          
1       2015-10-10         0      188        5     10   64      0.0   0.0   
        2015-10-11         0      272        6     10   68      6.0   0.0   
        2015-10-12         0      257        0     10   85      1.0   0.0   
        2015-10-13         0      239        1     10   84      5.0   0.0   
        2015-10-14         0      277        2     10  143      0.0   0.0   
        2015-10-15         0      227        3     10  164      6.0   0.0   
        2015-10-16         0      242        4     10  135      1.0   0.0   
        2015-10-17         0      293        5     10   78      5.0   0.0   
        2015-10-18         0      291        6     10   66      5.0   0.0   
        2015-10-19         0      256        0     10   67      0.0   0.0   
        2015-10-20         0      262        1     10   76      7.0   0.0   
        2015-10-21         0      271        2     10   69      0.0   0.0   
        2015-10-22         0      260        3     10   85      0.0   0.0   
        2015-10-23         0      240        4     10  102      7.0   0.0   
        2015-10-24         0      291        5     10   77      3.0   0.0   
        2015-10-25         0      286        6     10   71      7.0   0.0   
        2015-10-26         0      264        0     10   83     27.0   0.0   
        2015-10-27         0      295        1     10   66      8.0   0.0   
        2015-10-28         0      310        2     10   86     16.0   0.0   
        2015-10-29         0      212        3     10   76      4.0   0.0   
        2015-10-30         0      331        4     10   51      3.0   0.0   
        2015-10-31         0      347        5     10   40     16.0   0.0   
        2015-11-01         0      381        6     11   66      8.0   0.0   
        2015-11-02         0      332        0     11   80      5.0   0.0   
        2015-11-03         0      404        1     11  111      5.0   0.0   
        2015-11-04         0      392        2     11  112      3.0   0.0   
        2015-11-05         0      496        3     11   89      8.0   0.0   
        2015-11-06         0      430        4     11   63     16.0   0.0   
        2015-11-07         0      351        5     11   46      4.0   0.0   
        2015-11-08         0      288        6     11   42      4.0   0.0   
...                      ...      ...      ...    ...  ...      ...   ...   
2000    2016-10-02         4       39        6     10   65      0.0   2.0   
        2016-10-03         3       50        0     10   37      4.0   5.0   
        2016-10-04         8       29        1     10   46      0.0   1.0   
        2016-10-05         3       35        2     10   56      0.0   1.0   
        2016-10-06         4       76        3     10   54      4.0   5.0   
        2016-10-07        11       68        4     10   43      4.0   2.0   
        2016-10-08        16      158        5     10   36      2.0   5.0   
        2016-10-09        24      176        6     10   42      0.0   5.0   
        2016-10-10        35      161        0     10   42      3.0   2.0   
        2016-10-11        24      163        1     10   46      3.0   2.0   
        2016-10-12        33      159        2     10   52      0.0   2.0   
        2016-10-13        35      177        3     10   67      0.0   2.0   
        2016-10-14        14      148        4     10   58      2.0   2.0   
        2016-10-15         9       78        5     10   55      2.0   1.0   
        2016-10-16         4       60        6     10   60      2.0   2.0   
        2016-10-17        16      163        0     10   80      2.0   2.0   
        2016-10-18        18      157        1     10   61      2.0   1.0   
        2016-10-19        14      164        2     10   50      2.0   1.0   
        2016-10-20        35      165        3     10   40      4.0   2.0   
        2016-10-

In [3]:
df_usr_view_pay.columns

Index(['view_cnt', 'pay_cnt', 'weekday', 'month', 'aqi', 'weather', 'wind',
       'tmp'],
      dtype='object')

In [4]:
quant_features = ['view_cnt', 'pay_cnt', 'weekday', 'month', 'aqi', 'weather', 'wind', 'tmp']
# Store scalings in a dictionary so we can convert back later
scaled_features = {}
for each in quant_features:
    mean, std = df_usr_view_pay[each].mean(), df_usr_view_pay[each].std()
    scaled_features[each] = [mean, std]
    df_usr_view_pay.loc[:, each] = (df_usr_view_pay[each] - mean)/std
scaled_features

{'aqi': [69.10689907646486, 41.82863241974223],
 'month': [6.743497867987794, 3.1398223895350226],
 'pay_cnt': [108.25559585194213, 130.7345540633734],
 'tmp': [20.06643747988851, 9.000948791616628],
 'view_cnt': [15.710891994111511, 42.47333460898241],
 'weather': [12.96204816186815, 29.02202674421276],
 'weekday': [3.0034805507322524, 2.003086129482419],
 'wind': [1.3992824375980313, 2.446529545336942]}

In [5]:
# df_usr_view_pay = df_usr_view_pay.drop(['time_stamp', 'view_cnt'], axis=1)
df_usr_view_pay

view_cnt   pay_cnt   weekday     month       aqi  \
shop_id time_stamp                                                     
1       2015-10-10 -0.369900  0.609972  0.996722  1.037161 -0.122091   
        2015-10-11 -0.369900  1.252495  1.495951  1.037161 -0.026463   
        2015-10-12 -0.369900  1.137759 -1.499427  1.037161  0.379957   
        2015-10-13 -0.369900  1.000075 -1.000197  1.037161  0.356050   
        2015-10-14 -0.369900  1.290741 -0.500967  1.037161  1.766567   
        2015-10-15 -0.369900  0.908286 -0.001738  1.037161  2.268616   
        2015-10-16 -0.369900  1.023023  0.497492  1.037161  1.575311   
        2015-10-17 -0.369900  1.413126  0.996722  1.037161  0.212608   
        2015-10-18 -0.369900  1.397828  1.495951  1.037161 -0.074277   
        2015-10-19 -0.369900  1.130110 -1.499427  1.037161 -0.050370   
        2015-10-20 -0.369900  1.176004 -1.000197  1.037161  0.164794   
        2015-10-21 -0.369900  1.244846 -0.500967  1.037161 -0.002556   
        2015-10-22 -0.369900  1.160706 -0.001738  1.037161  0.379957   
        2015-10-23 -0.369900  1.007724  0.497492  1.037161  0.786378   
        2015-10-24 -0.369900  1.397828  0.996722  1.037161  0.188701   
        2015-10-25 -0.369900  1.359582  1.495951  1.037161  0.045258   
        2015-10-26 -0.369900  1.191303 -1.499427  1.037161  0.332143   
        2015-10-27 -0.369900  1.428424 -1.000197  1.037161 -0.074277   
        2015-10-28 -0.369900  1.543161 -0.500967  1.037161  0.403865   
        2015-10-29 -0.369900  0.793550 -0.001738  1.037161  0.164794   
        2015-10-30 -0.369900  1.703791  0.497492  1.037161 -0.432883   
        2015-10-31 -0.369900  1.826177  0.996722  1.037161 -0.695861   
        2015-11-01 -0.369900  2.086246  1.495951  1.355651 -0.074277   
        2015-11-02 -0.369900  1.711440 -1.499427  1.355651  0.260422   
        2015-11-03 -0.369900  2.262175 -1.000197  1.355651  1.001541   
        2015-11-04 -0.369900  2.170386 -0.500967  1.355651  1.025448   
        2015-11-05 -0.369900  2.965891 -0.001738  1.355651  0.475586   
        2015-11-06 -0.369900  2.461051  0.497492  1.355651 -0.145998   
        2015-11-07 -0.369900  1.856773  0.996722  1.355651 -0.552418   
        2015-11-08 -0.369900  1.374881  1.495951  1.355651 -0.648047   
...                      ...       ...       ...       ...       ...   
2000    2016-10-02 -0.275723 -0.529742  1.495951  1.037161 -0.098184   
        2016-10-03 -0.299268 -0.445602 -1.499427  1.037161 -0.767582   
        2016-10-04 -0.181547 -0.606233 -1.000197  1.037161 -0.552418   
        2016-10-05 -0.299268 -0.560338 -0.500967  1.037161 -0.313348   
        2016-10-06 -0.275723 -0.246726 -0.001738  1.037161 -0.361162   
        2016-10-07 -0.110914 -0.307919  0.497492  1.037161 -0.624139   
        2016-10-08  0.006807  0.380499  0.996722  1.037161 -0.791489   
        2016-10-09  0.195160  0.518183  1.495951  1.037161 -0.648047   
        2016-10-10  0.454146  0.403447 -1.499427  1.037161 -0.648047   
        2016-10-11  0.195160  0.418745 -1.000197  1.037161 -0.552418   
        2016-10-12  0.407058  0.388148 -0.500967  1.037161 -0.408976   
        2016-10-13  0.454146  0.525832 -0.001738  1.037161 -0.050370   
        2016-10-14 -0.040282  0.304008  0.497492  1.037161 -0.265533   
        2016-10-15 -0.158002 -0.231428  0.996722  1.037161 -0.337255   
        2016-10-16 -0.275723 -0.369111  1.495951  1.037161 -0.217719   
        2016-10-17  0.006807  0.418745 -1.499427  1.037161  0.260422   
        2016-10-18  0.053895  0.372850 -1.000197  1.037161 -0.193812   
        2016-10-19 -0.040282  0.426394 -0.500967  1.037161 -0.456790   
        2016-10-20  0.454146  0.434043 -0.001738  1.037161 -0.695861   
        2016-10-21 -0.063826  0.227518  0.497492  1.037161 -0.863210   
        2016-10-22 -0.228635 -0.323217  0.996722  1.037161 -1.150095   
        2016-10-23 -0.275723 -0.376760  1.495951  1.037161 -0.743675   
        2016-10-24  0.077439  0.411096 -1.499427  1.037161 -

In [6]:
df_shop_info = pd.read_csv('dataset/normed_shop_info_features.csv', index_col=[0])
df_shop_info

,city_idx,location_id,per_pay,score,comment_cnt,shop_level_0,shop_level_1,shop_level_2,cate_1_idx_0,cate_1_idx_1,...,cate_3_idx_34,cate_3_idx_35,cate_3_idx_36,cate_3_idx_37,cate_3_idx_38,cate_3_idx_39,cate_3_idx_40,cate_3_idx_41,cate_3_idx_42,cate_3_idx_43
1,1.192024,0.899196,-0.430197,1.397435e+00,2.742265e+00,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1.402179,-1.545979,1.478704,-1.501043e-14,4.805603e-15,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0.403944,0.568606,-0.950806,3.411693e-01,-3.495221e-01,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0.561560,-0.604840,1.305167,-1.501043e-14,4.805603e-15,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,-0.594291,-0.953299,-1.471415,-7.150959e-01,-3.495221e-01,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,0.666637,1.655681,0.437485,3.411693e-01,-6.587009e-01,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,-0.016366,-1.304737,0.437485,3.411693e-01,1.814729e+00,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
8,-0.594291,-1.039670,0.958095,3.411693e-01,-6.587009e-01,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,-0.594291,1.599093,0.784558,1.397435e+00,-9.678796e-01,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
10,1.664872,-1.444717,-0.083124,-1.501043e-14,4.805603e-15,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


# SPLIT DATA TO TWO SETS

In [7]:
validation_ratio = 0.1
num_samples = 2000

In [8]:
validation_idx = np.random.randint(1, num_samples+1, size=int(validation_ratio*num_samples))
print(validation_idx.shape)
validation_idx

(200,)


array([1734,   26, 1923, 1775,  809, 1918, 1534, 1566, 1338, 1929, 1114,
        137,  930,  277, 1346, 1471, 1112, 1093,  444,  338,  736,  891,
       1114,  294,  781,  371,   15,  358, 1665,  806, 1729, 1315,  421,
         54, 1544,  560, 1741,  762, 1252,   25, 1532,  210,  510, 1251,
       1316, 1111,  715, 1260,  869, 1296,  981,  962, 1472,  816,  558,
       1044,  333,  319,  164,  573, 1259, 1674, 1996,  366,  759,  444,
        476,  842,  499,  626,  252,   40, 1561, 1207, 1232, 1129,  879,
        493,  132,  735, 1037, 1856, 1895,  487, 1223, 1118,  620, 1314,
        529, 1308, 1928, 1462,  878,  444,  839, 1081, 1420, 1818, 1979,
       1577,  297, 1026,  650, 1034, 1991,  965,  396, 1162,  946, 1870,
       1839, 1265,  157, 1412,  804, 1249,  894, 1837, 1579, 1466, 1091,
        871,  237,  175, 1874,  224, 1024,  500, 1302,  620,  537,  624,
        528,  672,  886,  938,  359,  652, 1401, 1396, 1004, 1458,  253,
       1268,  989, 1155,  126,   76, 1293,  365,  5

In [9]:
len(set(validation_idx))

192

In [10]:
df_usr_train = df_usr_view_pay.drop(list(validation_idx), axis=0)
df_usr_train.reset_index(inplace=True)
df_usr_train.set_index(keys=['shop_id', 'time_stamp'], inplace=True)
df_usr_train

view_cnt   pay_cnt   weekday     month       aqi  \
shop_id time_stamp                                                     
1       2015-10-10 -0.369900  0.609972  0.996722  1.037161 -0.122091   
        2015-10-11 -0.369900  1.252495  1.495951  1.037161 -0.026463   
        2015-10-12 -0.369900  1.137759 -1.499427  1.037161  0.379957   
        2015-10-13 -0.369900  1.000075 -1.000197  1.037161  0.356050   
        2015-10-14 -0.369900  1.290741 -0.500967  1.037161  1.766567   
        2015-10-15 -0.369900  0.908286 -0.001738  1.037161  2.268616   
        2015-10-16 -0.369900  1.023023  0.497492  1.037161  1.575311   
        2015-10-17 -0.369900  1.413126  0.996722  1.037161  0.212608   
        2015-10-18 -0.369900  1.397828  1.495951  1.037161 -0.074277   
        2015-10-19 -0.369900  1.130110 -1.499427  1.037161 -0.050370   
        2015-10-20 -0.369900  1.176004 -1.000197  1.037161  0.164794   
        2015-10-21 -0.369900  1.244846 -0.500967  1.037161 -0.002556   
        2015-10-22 -0.369900  1.160706 -0.001738  1.037161  0.379957   
        2015-10-23 -0.369900  1.007724  0.497492  1.037161  0.786378   
        2015-10-24 -0.369900  1.397828  0.996722  1.037161  0.188701   
        2015-10-25 -0.369900  1.359582  1.495951  1.037161  0.045258   
        2015-10-26 -0.369900  1.191303 -1.499427  1.037161  0.332143   
        2015-10-27 -0.369900  1.428424 -1.000197  1.037161 -0.074277   
        2015-10-28 -0.369900  1.543161 -0.500967  1.037161  0.403865   
        2015-10-29 -0.369900  0.793550 -0.001738  1.037161  0.164794   
        2015-10-30 -0.369900  1.703791  0.497492  1.037161 -0.432883   
        2015-10-31 -0.369900  1.826177  0.996722  1.037161 -0.695861   
        2015-11-01 -0.369900  2.086246  1.495951  1.355651 -0.074277   
        2015-11-02 -0.369900  1.711440 -1.499427  1.355651  0.260422   
        2015-11-03 -0.369900  2.262175 -1.000197  1.355651  1.001541   
        2015-11-04 -0.369900  2.170386 -0.500967  1.355651  1.025448   
        2015-11-05 -0.369900  2.965891 -0.001738  1.355651  0.475586   
        2015-11-06 -0.369900  2.461051  0.497492  1.355651 -0.145998   
        2015-11-07 -0.369900  1.856773  0.996722  1.355651 -0.552418   
        2015-11-08 -0.369900  1.374881  1.495951  1.355651 -0.648047   
...                      ...       ...       ...       ...       ...   
2000    2016-10-02 -0.275723 -0.529742  1.495951  1.037161 -0.098184   
        2016-10-03 -0.299268 -0.445602 -1.499427  1.037161 -0.767582   
        2016-10-04 -0.181547 -0.606233 -1.000197  1.037161 -0.552418   
        2016-10-05 -0.299268 -0.560338 -0.500967  1.037161 -0.313348   
        2016-10-06 -0.275723 -0.246726 -0.001738  1.037161 -0.361162   
        2016-10-07 -0.110914 -0.307919  0.497492  1.037161 -0.624139   
        2016-10-08  0.006807  0.380499  0.996722  1.037161 -0.791489   
        2016-10-09  0.195160  0.518183  1.495951  1.037161 -0.648047   
        2016-10-10  0.454146  0.403447 -1.499427  1.037161 -0.648047   
        2016-10-11  0.195160  0.418745 -1.000197  1.037161 -0.552418   
        2016-10-12  0.407058  0.388148 -0.500967  1.037161 -0.408976   
        2016-10-13  0.454146  0.525832 -0.001738  1.037161 -0.050370   
        2016-10-14 -0.040282  0.304008  0.497492  1.037161 -0.265533   
        2016-10-15 -0.158002 -0.231428  0.996722  1.037161 -0.337255   
        2016-10-16 -0.275723 -0.369111  1.495951  1.037161 -0.217719   
        2016-10-17  0.006807  0.418745 -1.499427  1.037161  0.260422   
        2016-10-18  0.053895  0.372850 -1.000197  1.037161 -0.193812   
        2016-10-19 -0.040282  0.426394 -0.500967  1.037161 -0.456790   
        2016-10-20  0.454146  0.434043 -0.001738  1.037161 -0.695861   
        2016-10-21 -0.063826  0.227518  0.497492  1.037161 -0.863210   
        2016-10-22 -0.228635 -0.323217  0.996722  1.037161 -1.150095   
        2016-10-23 -0.275723 -0.376760  1.495951  1.037161 -0.743675   
        2016-10-24  0.077439  0.411096 -1.499427  1.037161 -

In [11]:
df_usr_train.index.levels[0]

Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            1988, 1989, 1990, 1992, 1993, 1994, 1997, 1998, 1999, 2000],
           dtype='int64', name='shop_id', length=1808)

In [12]:
df_usr_validation = df_usr_view_pay.loc[list(validation_idx)]
df_usr_validation.reset_index(inplace=True)
df_usr_validation.set_index(keys=['shop_id', 'time_stamp'], inplace=True)
df_usr_validation

view_cnt   pay_cnt   weekday     month       aqi  \
shop_id time_stamp                                                     
15      2016-07-08 -0.346356 -0.828056  0.497492  0.081693 -0.887117   
        2016-07-09 -0.369900 -0.828056  0.996722  0.081693 -0.313348   
        2016-07-10 -0.369900 -0.828056  1.495951  0.081693 -0.002556   
        2016-07-11 -0.369900 -0.828056 -1.499427  0.081693  0.069166   
        2016-07-12 -0.369900 -0.828056 -1.000197  0.081693 -0.815396   
        2016-07-13 -0.369900 -0.690373 -0.500967  0.081693 -0.432883   
        2016-07-14 -0.299268 -0.529742 -0.001738  0.081693 -0.982745   
        2016-07-15 -0.369900 -0.476198  0.497492  0.081693 -0.265533   
        2016-07-16 -0.346356 -0.384409  0.996722  0.081693 -0.145998   
        2016-07-17 -0.369900 -0.468549  1.495951  0.081693 -0.026463   
        2016-07-18 -0.369900 -0.437953 -1.499427  0.081693  0.021351   
        2016-07-19 -0.346356 -0.491497 -1.000197  0.081693 -0.934931   
        2016-07-20 -0.346356 -0.560338 -0.500967  0.081693 -0.217719   
        2016-07-21 -0.205091 -0.545040 -0.001738  0.081693  0.188701   
        2016-07-22 -0.369900 -0.407357  0.497492  0.081693 -0.122091   
        2016-07-23 -0.252179 -0.330866  0.996722  0.081693  0.284329   
        2016-07-24 -0.016737 -0.262024  1.495951  0.081693  0.140887   
        2016-07-25 -0.369900 -0.093744 -1.499427  0.081693 -0.887117   
        2016-07-26 -0.134458 -0.139639 -1.000197  0.081693 -0.624139   
        2016-07-27 -0.275723 -0.323217 -0.500967  0.081693 -0.671954   
        2016-07-28 -0.134458 -0.346164 -0.001738  0.081693  0.021351   
        2016-07-29 -0.299268 -0.231428  0.497492  0.081693  0.045258   
        2016-07-30 -0.181547  0.112781  0.996722  0.081693 -0.265533   
        2016-07-31 -0.299268  0.265763  1.495951  0.081693 -0.408976   
        2016-08-01 -0.205091 -0.254375 -1.499427  0.400183 -0.169905   
        2016-08-02 -0.228635 -0.185533 -1.000197  0.400183 -0.289440   
        2016-08-03 -0.299268 -0.177884 -0.500967  0.400183 -0.145998   
        2016-08-04 -0.299268 -0.262024 -0.001738  0.400183 -0.074277   
        2016-08-05 -0.181547 -0.208480  0.497492  0.400183 -0.169905   
        2016-08-06 -0.252179 -0.162586  0.996722  0.400183 -0.026463   
...                      ...       ...       ...       ...       ...   
1996    2016-10-02 -0.228635 -0.315568  1.495951  1.037161 -0.337255   
        2016-10-03 -0.110914 -0.376760 -1.499427  1.037161 -0.050370   
        2016-10-04 -0.275723 -0.384409 -1.000197  1.037161  0.140887   
        2016-10-05 -0.205091 -0.422655 -0.500967  1.037161 -0.002556   
        2016-10-06 -0.205091 -0.292620 -0.001738  1.037161  0.021351   
        2016-10-07 -0.110914 -0.307919  0.497492  1.037161  0.403865   
        2016-10-08  0.124528 -0.307919  0.996722  1.037161  0.427772   
        2016-10-09 -0.087370 -0.307919  1.495951  1.037161 -0.074277   
        2016-10-10 -0.181547 -0.239077 -1.499427  1.037161 -0.026463   
        2016-10-11 -0.205091 -0.177884 -1.000197  1.037161  0.069166   
        2016-10-12 -0.205091 -0.208480 -0.500967  1.037161  0.045258   
        2016-10-13 -0.063826 -0.147288 -0.001738  1.037161 -0.169905   
        2016-10-14 -0.087370 -0.193182  0.497492  1.037161 -0.385069   
        2016-10-15 -0.205091 -0.239077  0.996722  1.037161 -0.337255   
        2016-10-16 -0.134458 -0.254375  1.495951  1.037161 -0.217719   
        2016-10-17 -0.181547 -0.246726 -1.499427  1.037161 -0.241626   
        2016-10-18 -0.181547 -0.307919 -1.000197  1.037161 -0.408976   
        2016-10-19 -0.134458 -0.392059 -0.500967  1.037161 -1.221816   
        2016-10-20 -0.181547 -0.284971 -0.001738  1.037161 -1.030560   
        2016-10-21 -0.322812 -0.292620  0.497492  1.037161 -0.791489   
        2016-10-22 -0.322812 -0.177884  0.996722  1.037161 -0.385069   
        2016-10-23 -0.158002 -0.216130  1.495951  1.037161 -0.241626   
        2016-10-24 -0.252179 -0.162586 -1.499427  1.037161 -

In [13]:
len(df_usr_validation.index.levels[0])

192

# START LSTM~

### Define Data Generator

In [14]:
predict_days = 14
num_pred = 1  ## just predict pay_cnt

In [15]:
class Config(object):
    def __init__(self):
        self.num_steps = 60
        self.batch_size = 200
        self.lstm_size = 64
        self.lstm_layers = 2
        self.feature_size = 83
        self.keep_prob = 0.75
        self.target_size = 1  # pay count 
    def show(self):
        print('num_steps    : {}'.format(self.num_steps))
        print('batch_size   : {}'.format(self.batch_size))
        print('lstm_size    : {}'.format(self.lstm_size))
        print('lstm_layers  : {}'.format(self.lstm_layers))
        print('feature_size : {}'.format(self.feature_size))
        print('keep_prob    : {}'.format(self.keep_prob))
        print('target_size  : {}'.format(self.target_size))
config = Config()

In [16]:
# ss = df_usr_view_pay.loc[1]
# ss.reset_index(drop=True, inplace=True)
# ss

# num_s = 30
# sample1 = ss.loc[0:num_s-1].values
# print(sample1.shape)
# sss = [df_shop_info.loc[1].values]
# sss

# ssss = np.repeat(sss, 30, axis=0)
# ssss
# # np.equal(ssss[0, :], ssss[1, :])

# sample1.shape

# aaa = np.concatenate([ssss, sample1], axis=1)
# print(aaa.shape)
# aaa

In [17]:
def data_generator(config, df_usr, df_shop, random=True):
    batch_size = config.batch_size
    num_steps = config.num_steps
    target_size = config.target_size
    ID_list = list(df_usr.index.levels[0])
    m = len(ID_list)
    assert batch_size <= m, 'batch_size({}) > sample number({}), lower batch_size!'.format(batch_size, m)
    for batch_id in range(int(m/batch_size)): 
        if random:
            np.random.shuffle(ID_list)
            batch_ID_list = ID_list[0:batch_size]
        else:
            batch_ID_list = ID_list[batch_id*batch_size:(batch_id+1)*batch_size]
        _features, _labels = [], []
        for ID in batch_ID_list:
            each_shop_info = [df_shop.loc[ID].values]
            each_shop_usr = df_usr.loc[ID]
            each_shop_usr = each_shop_usr.reset_index(drop=True)
            _m = each_shop_usr.shape[0] - 1
            assert num_steps < _m, 'lower num_steps!'
            _s_idx = np.random.randint(0, _m-num_steps, 1)[0]
            _e_idx = _s_idx + num_steps
#             print('_m: {}, _s_idx: {}, _e_idx: {}'.format(_m, _s_idx, _e_idx))
            f1 = each_shop_usr.loc[_s_idx:_e_idx-1].values
            f1 = np.concatenate([np.repeat(each_shop_info, num_steps, axis=0), f1], axis=1)
            _features.append(f1)
            _labels.append(each_shop_usr.loc[_s_idx+1:_e_idx, 'pay_cnt'].values.reshape(-1, 1))
        
        features = np.stack(_features)
        labels = np.stack(_labels)
        yield features, labels, batch_ID_list


In [18]:
f, l, id_list = data_generator(config, df_usr_view_pay, df_shop_info).__next__()
print(f.shape)
print(l.shape)
print(id_list)

(200, 60, 83)
(200, 60, 1)
[1896, 1340, 1021, 5, 642, 1941, 114, 1079, 971, 1046, 1759, 847, 1219, 1608, 1627, 435, 85, 1066, 326, 1503, 573, 1306, 627, 876, 1469, 1735, 1862, 959, 733, 690, 1420, 1848, 1891, 1559, 8, 1188, 1585, 629, 1416, 1623, 434, 409, 912, 1353, 1354, 987, 852, 193, 1284, 1488, 1906, 1712, 1644, 1403, 1178, 1049, 1697, 1459, 1265, 112, 1, 621, 482, 499, 1241, 1959, 731, 1388, 142, 1673, 1077, 30, 998, 811, 75, 1822, 297, 539, 190, 1790, 641, 64, 1672, 440, 257, 835, 108, 1146, 1322, 1884, 1596, 1394, 311, 1498, 102, 595, 1849, 1181, 1413, 619, 330, 1161, 798, 1011, 1298, 807, 363, 1877, 879, 1206, 208, 870, 356, 358, 1537, 489, 125, 1217, 1620, 668, 78, 271, 1364, 1277, 1175, 1772, 133, 554, 1897, 1454, 1439, 1846, 1259, 667, 1114, 1541, 1902, 1160, 1099, 1531, 519, 796, 1407, 1809, 121, 1719, 940, 1643, 372, 1278, 1740, 1804, 1824, 367, 538, 457, 916, 1843, 1456, 674, 1035, 1176, 525, 1356, 760, 1395, 1998, 1484, 57, 1474, 34, 546, 436, 557, 763, 76, 1337, 786, 1

1. features shape: [batch_size, num_steps, feature_size]
2. labels shape: [batch_size, num_steps]

### Define LSTM

In [19]:
import tensorflow as tf

In [20]:
tf.__version__

'1.0.0'

In [21]:
config.show()

num_steps    : 60
batch_size   : 200
lstm_size    : 64
lstm_layers  : 2
feature_size : 83
keep_prob    : 0.75
target_size  : 1


In [37]:
class LstmModel(object):
    def __init__(self, config, is_training):
        tf.reset_default_graph()
        self._bath_size = batch_size = config.batch_size
        self._feature_size = feature_size = config.feature_size
        self._num_steps = num_steps = config.num_steps
        self._target_size = target_size = config.target_size
        self._lstm_size = lstm_size = config.lstm_size
        self._keep_prob = keep_prob = config.keep_prob
        self._lstm_layers = lstm_layers = config.lstm_layers
        
        self._x = x = tf.placeholder(tf.float32, shape=[batch_size, num_steps, feature_size])
        self._y = y = tf.placeholder(tf.float32, shape=[batch_size, num_steps, target_size])
        
        lstm_cell = tf.contrib.rnn.LSTMCell(lstm_size, forget_bias=0.1)
        if is_training and keep_prob < 1:
            lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=keep_prob)
        lstm_cell = tf.contrib.rnn.MultiRNNCell([lstm_cell] * lstm_layers)
        
        self._initial_state = lstm_cell.zero_state(batch_size, tf.float32)
        
        if is_training and keep_prob < 1:
            x = tf.nn.dropout(x, keep_prob)
        
        outputs = []
        state = self._initial_state
        with tf.variable_scope('lstm'):
            for step in range(num_steps):
                if step > 0:
                    tf.get_variable_scope().reuse_variables()
                cell_output, state = lstm_cell(x[:, step, :], state)
                outputs.append(cell_output)
        self._final_state = state
        outputs = tf.reshape(tf.concat(outputs, axis=1), [-1, lstm_size])
        
        reg_w = tf.get_variable('reg_w', [lstm_size, target_size])
        reg_b = tf.get_variable('reg_b', [target_size])
        self._reg_out = reg_out = tf.matmul(outputs, reg_w) + reg_b
        
#         loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([reg_out], [tf.reshape(y, [-1])], 
#                                                                   [tf.ones([batch_size * num_steps])])
#         self._loss = loss = tf.reduce_sum(loss) / batch_size
        self._loss = loss = tf.reduce_mean(tf.pow(tf.subtract(tf.reshape(reg_out, [-1]),
                                                 tf.reshape(y, [-1])), 2), name='loss')
        if not is_training:
            return

        self._optimizer = tf.train.AdamOptimizer().minimize(loss)

    @property
    def x(self):
        return self._x
    
    @property
    def y(self):
        return self._y

    @property
    def batch_size(self):
        return self._bath_size

    @property
    def feature_size(self):
        return self._feature_size
    
    @property
    def num_steps(self):
        return self._num_steps
    
    @property
    def optimizer(self):
        return self._optimizer
    
    @property
    def initial_state(self):
        return self._initial_state
    
    @property
    def final_state(self):
        return self._final_state
    
    @property
    def loss(self):
        return self._loss
    
    @property
    def reg_out(self):
        return self._reg_out
        
        

In [88]:
epochs = 50
keep_probablity = 0.75
config.batch_size = 100

In [89]:
config.show()
m = LstmModel(config, is_training=True)

num_steps    : 60
batch_size   : 100
lstm_size    : 64
lstm_layers  : 2
feature_size : 83
keep_prob    : 0.75
target_size  : 1


In [90]:
save_model_path = 'model/customer_prediction'
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        new_state = sess.run(m.initial_state)
        for batch_id, (features, labels, _) in enumerate(data_generator(config, df_usr_view_pay, df_shop_info)):
            train_feed_dict = {m.x: features,
                               m.y: labels,
                               m.initial_state: new_state}
            loss, new_state, _ = sess.run([m.loss, m.final_state, m.optimizer], feed_dict=train_feed_dict)
            print('epoch: {:>2}, batch: {:>2}, train_loss: {:>8.4f}'.format(epoch+1, batch_id+1, loss))
    # Save Model
    saver = tf.train.Saver()
    save_path = saver.save(sess, save_model_path)


epoch:  1, batch:  1, train_loss:   0.5828
epoch:  1, batch:  2, train_loss:   3.9615
epoch:  1, batch:  3, train_loss:   2.6853
epoch:  1, batch:  4, train_loss:   1.0823
epoch:  1, batch:  5, train_loss:   0.3268
epoch:  1, batch:  6, train_loss:   0.5564
epoch:  1, batch:  7, train_loss:   0.6193
epoch:  1, batch:  8, train_loss:   1.1970
epoch:  1, batch:  9, train_loss:   0.5593
epoch:  1, batch: 10, train_loss:   0.7766
epoch:  1, batch: 11, train_loss:   0.5981
epoch:  1, batch: 12, train_loss:   0.4731
epoch:  1, batch: 13, train_loss:   0.8415
epoch:  1, batch: 14, train_loss:   0.3801
epoch:  1, batch: 15, train_loss:   0.2931
epoch:  1, batch: 16, train_loss:   0.3177
epoch:  1, batch: 17, train_loss:   0.3572
epoch:  1, batch: 18, train_loss:   0.4335
epoch:  1, batch: 19, train_loss:   0.2954
epoch:  1, batch: 20, train_loss:   0.2487
epoch:  2, batch:  1, train_loss:   0.2536
epoch:  2, batch:  2, train_loss:   0.4679
epoch:  2, batch:  3, train_loss:   0.3060
epoch:  2, 

# CHECKPOINT

In [91]:
each = df_usr_view_pay.loc[1]
each = each.reset_index(drop=True)
each
each.iloc[-1]

view_cnt   -0.040282
pay_cnt     0.548779
weekday    -1.499427
month       1.037161
aqi        -0.743675
weather    -0.377715
wind        0.245539
tmp        -0.729527
Name: 387, dtype: float64

In [97]:
def test_data_generator(config, df_usr, df_shop, final=False):
    batch_size = config.batch_size
    ID_list = list(df_usr.index.levels[0])
    m = len(ID_list)
    assert batch_size <= m, 'batch_size({}) > sample number({}), lower batch_size!'.format(batch_size, m)
    for batch_id in range(int(m/batch_size)): 
        batch_ID_list = ID_list[batch_id*batch_size:(batch_id+1)*batch_size]
        _features, _labels = [], []
        for ID in batch_ID_list:
            each_shop_info = [df_shop.loc[ID].values]
            each_shop_usr = df_usr.loc[ID]
            if final:
                f1 = each_shop_usr.iloc[-1].values.reshape(1, -1)
                f1 = np.concatenate([np.repeat(each_shop_info, num_steps, axis=0), f1], axis=1)
                _features.append(f1)
            else:
                each_shop_usr = each_shop_usr.reset_index(drop=True)
                _m = each_shop_usr.shape[0] - predict_days
                assert num_steps < _m, 'lower num_steps!'
                _idx = np.random.randint(0, _m, 1)[0]
                f1 = each_shop_usr.loc[_idx].values.reshape(1, -1)
                f1 = np.concatenate([each_shop_info, f1], axis=1)
                _features.append(f1)
                _labels.append(each_shop_usr.loc[_idx+1:_idx+predict_days, 'pay_cnt'].values.reshape(-1, 1))
        last_feature = np.stack(_features)
        if final:
            yield last_feature, batch_ID_list
        else:
            gt = np.stack(_labels)
            yield last_feature, gt, batch_ID_list

In [98]:
test_config = Config()
test_config.num_steps = 1
test_config.batch_size = 100
test_config.keep_prob = 1
test_config.show()

num_steps    : 1
batch_size   : 100
lstm_size    : 64
lstm_layers  : 2
feature_size : 83
keep_prob    : 1
target_size  : 1


In [99]:
m_test = LstmModel(test_config, is_training=False)

In [95]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import tensorflow as tf

save_model_path = 'model/customer_prediction'

with tf.Session() as sess:
    
    saver = tf.train.Saver()
    saver.restore(sess, save_model_path)

    # Get accuracy in batches for memory limitations
    test_batch_loss_total = 0
    test_batch_count = 0
    result_list = []
    reg_out_list = []
    new_state = sess.run(m_test.initial_state)
    for features, labels, batch_ID_list in test_data_generator(test_config, df_usr_view_pay, df_shop_info):
        for day in 
        test_feed_dict={m_test.x: features, m_test.y: labels, m_test.initial_state: new_state}
        test_batch_loss_total += sess.run(m_test.loss, feed_dict=test_feed_dict)
        test_batch_count += 1
        test_reg_out = sess.run(m_test.reg_out, feed_dict=test_feed_dict)
        tmp_list = [pd.DataFrame(np.array([test_reg_out[i,:], labels[i,:]]).T, columns=['pred', 'label']) for i in range(test_config.batch_size)]
        result_list.append(pd.concat(tmp_list, keys=batch_ID_list, names=['shop_id', 'predict_day']))
        reg_out_list.append(pd.DataFrame(test_reg_out))
        
    df_result = pd.concat(result_list)
    
    final_reg_out_list = []
    for features, batch_ID_list in test_data_generator(test_config, df_usr_view_pay, df_shop_info, final=True):
        test_feed_dict={m_test.x: features, m_test.initial_state: new_state}
        final_reg_out = sess.run(m_test.reg_out, feed_dict=test_feed_dict)
        final_reg_out_list.append(pd.DataFrame(final_reg_out))
   
    df_pred = pd.concat(final_reg_out_list, ignore_index=True)
    

    print('Testing Cost: {}\n'.format(test_batch_loss_total/test_batch_count))


Testing Cost: 0.28759882897138594



In [96]:
df_result

,,pred,label
shop_id,predict_day,,
1,0,-0.007964,0.464639
2,0,0.572622,1.459021
3,0,-0.258811,-0.284971
4,0,0.067097,0.059238
5,0,-0.480833,-0.828056
6,0,-0.299275,-0.147288
7,0,-0.108466,-0.262024
8,0,-0.258568,-0.369111
9,0,0.136650,-0.070797


In [49]:
def negtive_check(df, op=True):
    a = unnormed_df_pred < 0
    if a.sum().sum() > 0:
        print('Find negative!')
        if op:
            df[a] = 0
    else:
        print('no negative number~')

In [ ]:
df_result

In [ ]:
scaled_features

In [ ]:
mean, std = scaled_features['pay_cnt']
unnormed_df_result = df_result *std + mean
unnormed_df_result

In [ ]:
_pred = unnormed_df_result.pred.values
_label = unnormed_df_result.label.values
loss = np.mean(np.abs((_pred-_label)/(_pred+_label)))
loss

### OUTPUT PREDICT

In [ ]:
mean, std = scaled_features['pay_cnt']
unnormed_df_pred = df_pred *std + mean
unnormed_df_pred

In [ ]:
negtive_check(unnormed_df_pred)
unnormed_df_pred


In [ ]:

int_df_pred = unnormed_df_pred.apply(np.round).astype(np.int16)
int_df_pred

int_df_pred.insert(value=range(1, 2001), loc=0, column='id')
int_df_pred

int_df_pred.to_csv('pred/pred_2.csv', header=False, index=False)
int_df_pred

In [ ]:
last_half_month = df_usr_view_pay.loc[(slice(None), slice('2016-10-17', '2016-10-31')), ('pay_cnt')]
mean, std = scaled_features['pay_cnt']
last_half_month = last_half_month * std + mean
last_half_month